In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [10]:
def fn(df, threshold):
    return pd.DataFrame(df.groupby(['Sampling Period', 'Site Name', 'Sampling Unit Name','Genus', 'Species']).apply(lambda x: x[x['Time Between Photos'] < threshold]['Number of Animals'].max()), columns = ['Number of Animals']).reset_index()

In [91]:
def fn_marin(df, threshold):
    return pd.DataFrame(df.groupby(['Sampling Period', 'Project ID.1', 'Deployment Location ID', 'Genus', 'Species']).apply(lambda x: x[x['Time Between Photos'] < threshold]['Count'].max()), columns = ['Number of Animals']).reset_index()

In [8]:
team = pd.read_csv('../data/Terrestrial_Vertebrate.csv', index_col=0)

/Users/terencekwt/Code/datadive_201608_ci/env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10,19,20,21,26,27,29,30,31,32,34,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
team.columns

Index([u'Site Name', u'Sampling Unit Name', u'Latitude', u'Longitude',
       u'Sampling Period', u'Photo Type', u'Photo Date', u'Photo Time',
       u'Raw Name', u'Photo Notes', u'Photo ID URL', u'Class', u'Order',
       u'Family', u'Genus', u'Species', u'Number of Animals',
       u'Person Identifying the Photo', u'Camera Serial Number',
       u'Memory Card Serial Number', u'Camera Notes',
       u'Camera Start Date and Time', u'Camera End Date and Time',
       u'Person setting up the Camera', u'Person picking up the Camera',
       u'Camera Make', u'Camera Model', u'Flash', u'Exposure Time',
       u'Sequence Info', u'Triplet Number', u'Moon Phase', u'Temperature',
       u'Camera Firmware Version', u'Protocol Version',
       u'Data Set Creator Institutions', u'Data Set Creator Scientist',
       u'Data Set Contact', u'Data Level'],
      dtype='object')

In [11]:
team = team.sort_values(['Site Name', 'Sampling Unit Name', 'Genus', 'Species', 'Photo Time'])
team = team.reset_index()

In [13]:
team.columns

Index([u'ID', u'Site Name', u'Sampling Unit Name', u'Latitude', u'Longitude',
       u'Sampling Period', u'Photo Type', u'Photo Date', u'Photo Time',
       u'Raw Name', u'Photo Notes', u'Photo ID URL', u'Class', u'Order',
       u'Family', u'Genus', u'Species', u'Number of Animals',
       u'Person Identifying the Photo', u'Camera Serial Number',
       u'Memory Card Serial Number', u'Camera Notes',
       u'Camera Start Date and Time', u'Camera End Date and Time',
       u'Person setting up the Camera', u'Person picking up the Camera',
       u'Camera Make', u'Camera Model', u'Flash', u'Exposure Time',
       u'Sequence Info', u'Triplet Number', u'Moon Phase', u'Temperature',
       u'Camera Firmware Version', u'Protocol Version',
       u'Data Set Creator Institutions', u'Data Set Creator Scientist',
       u'Data Set Contact', u'Data Level'],
      dtype='object')

In [14]:
team['NEW_PHOTO_TIME'] = team['Photo Date'] + ' ' + team['Photo Time']
team['NEW_PHOTO_TIME'] = pd.to_datetime(team['NEW_PHOTO_TIME'])

In [15]:
team['Time Between Photos'] = team['NEW_PHOTO_TIME'] - team['NEW_PHOTO_TIME'].shift()
team['Time Between Photos'].fillna(timedelta(0), inplace=True)


team['Species'].dropna(inplace=True)

team['Time Between Photos'] = team['Time Between Photos'].astype('timedelta64[s]')
team['Time Between Photos'] = team['Time Between Photos'].apply(lambda x: 0 if x > 100 else x)

In [16]:
team_new = fn(team, 3)

In [19]:
team_new.head()

,Sampling Period,Site Name,Sampling Unit Name,Genus,Species,Number of Animals
0,2007.01,Volcán Barva,CT-VB-1-1,Cuniculus,paca,1.0
1,2007.01,Volcán Barva,CT-VB-1-1,Dasyprocta,punctata,1.0
2,2007.01,Volcán Barva,CT-VB-1-1,Dasypus,novemcinctus,1.0
3,2007.01,Volcán Barva,CT-VB-1-1,Didelphis,marsupialis,1.0
4,2007.01,Volcán Barva,CT-VB-1-1,Homo,sapiens,1.0


In [20]:
team_new.to_csv('../data/Max_Animals_TEAM_data.csv')

In [49]:
marin = pd.read_csv('../data/Marin_Merged.csv', index_col=0)

In [50]:
marin.columns

Index([u'Deployment ID', u'Event', u'Array Name', u'Deployment Location ID',
       u'Longitude Resolution', u'Latitude Resolution',
       u'Camera Deployment Begin Date', u'Camera Deployment End Date',
       u'Bait Type', u'Bait Description', u'Feature Type',
       u'Feature Type methodology', u'Camera ID', u'Quiet Period Setting',
       u'Restriction on access', u'Camera Failure Details', u'Make', u'Model',
       u'Serial Number', u'Year Purchased', u'Project ID.1', u'Image ID',
       u'Location', u'Photo Type', u'Photo Type Identified by', u'Uncertainty',
       u'IUCN Identification Number', u'Date_Time Captured', u'Age', u'Sex',
       u'Individual ID', u'Count', u'Animal recognizable (Y/N)',
       u'individual Animal notes', u'Genus', u'Species'],
      dtype='object')

In [61]:
print marin["Date_Time Captured"][0]
print pd.to_datetime(marin["Date_Time Captured"][0], format = "%Y-%m-%d %H:%M:%S")

2014-09-22 08:12:40
2014-09-22 08:12:40


In [65]:
# 2015-04-01 22:46:18 PM

type(marin["Date_Time Captured"][1])

str

In [76]:
# marin["Date_Time Captured"].apply(lambda x: x.replace("PM", ""))
# marin["Date_Time Captured"].apply(lambda x: x.replace("AM", ""))

In [77]:
marin['NEW_PHOTO_TIME'] = pd.to_datetime(marin['Date_Time Captured'], format = "%Y-%m-%d %H:%M:%S")

In [78]:
marin['Time Between Photos'] = marin['NEW_PHOTO_TIME'] - marin['NEW_PHOTO_TIME'].shift()
marin['Time Between Photos'].fillna(timedelta(0), inplace=True)

In [79]:
marin = marin[marin['Species'] != 'unknown']

In [80]:
marin['Species'].dropna(inplace=True)

In [83]:
marin['Time Between Photos'] = marin['Time Between Photos'].astype('timedelta64[s]')
marin['Time Between Photos'] = marin['Time Between Photos'].apply(lambda x: 0 if x > 100 else x)

In [85]:
marin['Date_Time Captured'].dropna(inplace=True)
marin['Sampling Period'] = marin['Date_Time Captured'].apply(lambda x: x[0:7])

In [90]:
marin.columns

Index([u'Deployment ID', u'Event', u'Array Name', u'Deployment Location ID',
       u'Longitude Resolution', u'Latitude Resolution',
       u'Camera Deployment Begin Date', u'Camera Deployment End Date',
       u'Bait Type', u'Bait Description', u'Feature Type',
       u'Feature Type methodology', u'Camera ID', u'Quiet Period Setting',
       u'Restriction on access', u'Camera Failure Details', u'Make', u'Model',
       u'Serial Number', u'Year Purchased', u'Project ID.1', u'Image ID',
       u'Location', u'Photo Type', u'Photo Type Identified by', u'Uncertainty',
       u'IUCN Identification Number', u'Date_Time Captured', u'Age', u'Sex',
       u'Individual ID', u'Count', u'Animal recognizable (Y/N)',
       u'individual Animal notes', u'Genus', u'Species', u'NEW_PHOTO_TIME',
       u'Time Between Photos', u'Sampling Period'],
      dtype='object')

In [92]:
marin_new = fn_marin(marin, 3)

In [93]:
marin_new.to_csv('../data/Max_Animals_MARIN_data.csv')